# CCN Assignment 6: Generative	Adversarial	Networks

In [1]:
%matplotlib inline

import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.dataset import concat_examples
from chainer.datasets import TupleDataset
from chainer.functions.evaluation import accuracy
from chainer.functions.loss import softmax_cross_entropy
from chainer import link
from chainer import reporter

import matplotlib.pyplot as plt
from utils import get_mnist

In [5]:
# make loss function such that loss is high when discriminator 
# classifies generator network output as fake 0, and
# low when generator network output as being a real 0

class Generator(Chain):
    def __init__(self, n_units, n_output):
        super(Generator, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(n_units, n_units) # fully connected layer
            self.batch_norm = L.BatchNormalization(n_units) # batch normalization
            self.deconv = L.Deconvolution2D(None, n_output) # deconvolution layer
            
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = self.batch_norm(h1)
        h3 = self.deconv(h2)
        y = F.sigmoid(h3)
        return y

In [3]:
class Discriminator(Chain):
    def __init__(self, n_out):
        super(Discriminator, self).__init__()
        with self.init_scope():
            self.conv = L.Convolution2D(None, out_channels=5, ksize=5, stride=1, pad=0)
            self.fc1 = L.Linear(None, n_out)
            
        def __call__(self, x):
            h1 = self.conv1(x)
            h2 = F.max_pooling_2d(h1, ksize=5, stride=1, pad=0) # max pooling layer
            y = F.relu(self.fc1(h2))
            return y

In [6]:
class Regressor(link.Chain):

    compute_accuracy = False

    def __init__(self, predictor,
                 lossfun=softmax_cross_entropy.softmax_cross_entropy,
                 accfun=accuracy.accuracy,
                 label_key=-1):
        if not (isinstance(label_key, (int, str))):
            raise TypeError('label_key must be int or str, but is %s' %
                            type(label_key))

        super(Regressor, self).__init__()
        self.lossfun = lossfun
        self.accfun = accfun
        self.y = None
        self.loss = None
        self.accuracy = None
        self.label_key = label_key

        with self.init_scope():
            self.predictor = predictor

    def __call__(self, *args, **kwargs):

        if isinstance(self.label_key, int):
            if not (-len(args) <= self.label_key < len(args)):
                msg = 'Label key %d is out of bounds' % self.label_key
                raise ValueError(msg)
            t = args[self.label_key]
            if self.label_key == -1:
                args = args[:-1]
            else:
                args = args[:self.label_key] + args[self.label_key + 1:]
        elif isinstance(self.label_key, str):
            if self.label_key not in kwargs:
                msg = 'Label key "%s" is not found' % self.label_key
                raise ValueError(msg)
            t = kwargs[self.label_key]
            del kwargs[self.label_key]

        self.y = None
        self.loss = None
        self.accuracy = None
        self.y = self.predictor(*args, **kwargs)
        self.loss = self.lossfun(self.y, t)
        reporter.report({'loss': self.loss}, self)
        return self.loss

In [7]:
def run(generator, discriminator, regressor, classifier, gen_optimizer, discr_optimizer, true_train_iter, true_val_iter, noise_train_iter):
#     First generate a sample with the generator network.
#     Classify it with the discriminator network, 
#     calculate the loss such that you enhance samples that the discriminator thinks are real 
#     and update the networks. 
#     Next calculate the loss of the generated sample enhancing those samples 
#     that the discriminator correctly recognizes as fake. 
#     Combine this with the loss that the discriminator gets on real images 
#     and update the networks based on this combined loss.
    
    # get next mini-batch from generator
    noise_batch = noise_train_iter.next()
    # generate batch
    gen = generator(noise_batch)
    # classify
    clas = discriminator(gen)
    # compute loss
    label = np.array([0])
    discr_loss = classifier(clas, label)
    # compute gradients
    classifier.cleargrads()
    discr_loss.backward()
    # update variables
    discr_optimizer.update()
    
    # loss of generated sample
    gen_loss = 1-discr_loss
    # compute gradients
    regressor.cleargrads()
    gen_loss.backward()
    # update variables
    gen_optimizer.update()
    
    # get next mini-batch from real data
    real_batch = true_train_iter.next()
    # classify
    clas = discriminator(gen)
    # compute loss
    label = np.array([1])
    discr_loss = classifier(clas, label)
    # compute gradients
    classifier.cleargrads()
    discr_loss.backward()
    # update variables
    discr_optimizer.update()

In [8]:
def main():
    # generate the real data
    train_data, test_data = get_mnist(n_train=1000, n_test=100, with_label=False, classes = [0])
    
    # generate the fake data
    noise_train = np.random.rand(1000,28,28)
    fake_test = np.random.rand(100,28,28)
    
    # initialize interators
    true_train_iter = iterators.SerialIterator(train_data, batch_size=64, shuffle=True)
    true_val_iter = iterators.SerialIterator(test_data, batch_size=100, repeat=False, shuffle=False)
    noise_train_iter = iterators.SerialIterator(noise_train, batch_size=64, shuffle=True)
    
    # define generator
    generator = Generator(784, 784)
    regressor = Regressor(generator, F.mean_squared_error)
    gen_optimizer = optimizers.SGD()
    gen_optimizer.setup(regressor)
    
    # define discriminator
    discriminator = Discriminator(1)
    classifier = L.Classifier(discriminator, F.sigmoid_cross_entropy)
    discr_optimizer = optimizers.SGD()
    discr_optimizer.setup(classifier)
    
    run(generator, discriminator, regressor, classifier, gen_optimizer, discr_optimizer, true_train_iter, true_val_iter, noise_train_iter)
    
if __name__ == "__main__":
    main()

TypeError: numpy.ndarray or cuda.ndarray are expected.
Actual: <type 'list'>